This example is based on the example in section 5.4.3, in selecting a kernel function that makes sense for the Mauna Loa $CO_2$ data. Similar concepts can be applied to our dataset of bike data over time.

In [1]:
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ExpSineSquared, RationalQuadratic, WhiteKernel

# Dataset

`sklearn` provides us with the dummy dataset used for this example.

In [2]:
data = fetch_mldata('mauna-loa-atmospheric-co2').data
X = data[:, [1]]
y = data[:, 0]

# Covariance Function Definition

We follow closely the covariance functions defined in RW, and we learn them with our Maximum Likelihood Estimation.

### Long term smooth rising trend

We make use the a squared exponential covariance term with **two** hyperparameters controlling amplitude $\theta_1$ and characteristic length-scale $\theta_2$.

$$k_1(x, x') = \theta_1^2 \text{exp}\Big( - \frac{(x - x')^2}{2 \theta_2^2} \Big)$$

In [3]:
def k1(theta1, theta2):
    return theta1**2 * RBF(length_scale=theta2)

### Decay from exact periodicity

The seasonal trend of $CO_2$ concentration is not exactly periodic. We can take the product with another RBF to allow a *decay away* from the exact periodicity.

$$k_2(x, x') = \theta_3^2 \text{exp} \Big(- \frac{(x - x')^2}{2 \theta_4^2} - \frac{2 \text{sin}^2 (\pi (x - x'))}{\theta_5^2} \Big)$$

In [4]:
def k2(theta3, theta4, theta5):
    return theta3**2 * RBF(length_scale=theta4) \
        * ExpSineSquared(length_scale=theta5, periodicity=1.0,
                         periodicity_bounds="fixed") # seasonal component

### Medium term irregularities

(I am not very sure what is this)

A rational quadratic term is used here.

$$k_3(x, x') = \theta_6^2 \Big( 1 + \frac{(x - x')^2}{2\theta_8 \theta_7^2} \Big)^{- \theta_8 }$$

In [5]:
def k3(theta6, theta7, theta8):
    return theta6**2 * RationalQuadratic(length_scale=theta7, alpha=theta8)

### Noise Model

We model the noise as the sum of a squared exponential contribution and an independent component.

$$k4(x_p, x_q) = \theta_9^2 \text{exp} \Big( - \frac{(x_p - x_q)^2}{2 \theta_10^2} \Big) + \theta_{11}^2\delta_{pq}$$

In [6]:
def k4(theta9, theta10, theta11):
    return 0.1**2 * RBF(length_scale=0.1) \
        + WhiteKernel(noise_level=0.1**2,
                      noise_level_bounds=(1e-3, np.inf)) # noise terms

---

Our final covariance function is simply the sum of all covariance functions.

$$k(x, x') = k_1(x, x') + k_2(x, x') + k_3(x, x') + k_4(x, x'),$$

with $[\theta_1 \ldots \theta_{11}]$ hyperparameters to optimize.

---

In [7]:
def k_all(thetas):
    return k1(thetas[0], thetas[1]) + k2(thetas[2], thetas[3], thetas[4])\
        + k3(thetas[5], thetas[6], thetas[7]) + k4(thetas[8], thetas[9], thetas[10])

# Model

We are going to make use of `sklearn`'s Gaussian Process model to fit our data with the defined covariance function. Let's first try out random $\theta$ and see how it goes.

## Helper methods to know what's happening

In [8]:
def print_likelihood(gp):
    print("\nLearned kernel: %s" % gp.kernel_)
    print("Log-marginal-likelihood: %.3f"
          % gp.log_marginal_likelihood(gp.kernel_.theta))

### Initialize random hyperparameters

In [9]:
thetas = np.random.rand(11).tolist()

### Randomized Parameters

Just for the fun of it, let's see how we perform with random hyperparameters.

In [10]:
gp = GaussianProcessRegressor(kernel=k_all(thetas), alpha=0, normalize_y=True, optimizer=None)

In [11]:
gp.fit(X, y)

GaussianProcessRegressor(alpha=0, copy_X_train=True,
             kernel=0.868**2 * RBF(length_scale=0.525) + 0.934**2 * RBF(length_scale=0.369) * ExpSineSquared(length_scale=0.215, periodicity=1) + 0.855**2 * RationalQuadratic(alpha=0.258, length_scale=0.681) + 0.1**2 * RBF(length_scale=0.1) + WhiteKernel(noise_level=0.01),
             n_restarts_optimizer=0, normalize_y=True, optimizer=None,
             random_state=None)

In [12]:
print_likelihood(gp)


Learned kernel: 0.868**2 * RBF(length_scale=0.525) + 0.934**2 * RBF(length_scale=0.369) * ExpSineSquared(length_scale=0.215, periodicity=1) + 0.855**2 * RationalQuadratic(alpha=0.258, length_scale=0.681) + 0.1**2 * RBF(length_scale=0.1) + WhiteKernel(noise_level=0.01)
Log-marginal-likelihood: -2455.752


Yeah that sucks, turns out that `sklearn`'s GPR has a neat little feature that bakes in hyperparameter optimization within the fitting process. Let's try that.

### Optimized Parameters

Turns out that the `optimizer` kwarg defaults to `fmin_l_bfgs_b`. Which I have no idea what that is. Need to read a bit more about this. But either way, it is tuning the hyperparameters to maximize our log marginal likelihood.

In [13]:
gp = GaussianProcessRegressor(kernel=k_all(thetas), alpha=0, normalize_y=True)

In [14]:
gp.fit(X, y)

GaussianProcessRegressor(alpha=0, copy_X_train=True,
             kernel=0.868**2 * RBF(length_scale=0.525) + 0.934**2 * RBF(length_scale=0.369) * ExpSineSquared(length_scale=0.215, periodicity=1) + 0.855**2 * RationalQuadratic(alpha=0.258, length_scale=0.681) + 0.1**2 * RBF(length_scale=0.1) + WhiteKernel(noise_level=0.01),
             n_restarts_optimizer=0, normalize_y=True,
             optimizer='fmin_l_bfgs_b', random_state=None)

In [15]:
print_likelihood(gp)


Learned kernel: 0.442**2 * RBF(length_scale=0.951) + 3.27**2 * RBF(length_scale=181) * ExpSineSquared(length_scale=1.44, periodicity=1) + 34.4**2 * RationalQuadratic(alpha=1.45e+04, length_scale=41.7) + 0.198**2 * RBF(length_scale=0.138) + WhiteKernel(noise_level=0.0336)
Log-marginal-likelihood: -83.221


`-83.214` is better than the result that the textbook gave, which is around `-108.5`.

# Short Conclusion

The `GaussianProcessRegressor` provided by `sklearn` is pretty simple to use. The kernel definition is pretty customizable for our usage as well. 

The convenience brought about by the `optimizer` also cuts down a lot of boilerplate code that we might have to write to run hyperparameter optimization.

Here are several parts that we should explore a little bit more:

- **Validation**: how do we know that what we've predicted is accurate?
- **Visualizations**: plotting of the graph

Validation is an important step because we want to ensure that we know our model predicted is reliable for our planning purposes in the second part of the project.